# ASL Recognition: End-to-End Workflow

This notebook consolidates the entire workflow for preprocessing, model training, and evaluation for American Sign Language (ASL) recognition.


In [1]:
# Import Required Libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Preprocessing: Load and Normalize Keypoints

We will load the preprocessed keypoints from the JSON files and normalize them.


In [ ]:
# Configuration
DATA_DIR = Path(r'f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints')

# Load and normalize keypoints
def load_keypoints(data_dir):
    data = []
    labels = []
    skipped_files = 0
    processed_files = 0
    
    print(f"Loading keypoints from {data_dir}")
    
    for label_dir in tqdm(data_dir.iterdir(), desc="Loading labels"):
        if not label_dir.is_dir():
            continue
        
        label_files = list(label_dir.glob('*.json'))
        print(f"Found {len(label_files)} files in {label_dir.name}")
        
        for json_file in tqdm(label_files, desc=f"Processing {label_dir.name}", leave=False):
            try:
                with open(json_file, 'r') as f:
                    content = json.load(f)
                    
                    # Ensure required keys are present
                    if 'keypoints' not in content or 'label' not in content:
                        print(f"Warning: Missing required keys in {json_file}")
                        skipped_files += 1
                        continue
                    
                    keypoints = content['keypoints']
                    label = content['label']
                    
                    # Process the keypoints into a consistent format
                    processed_frames = []
                    
                    for frame in keypoints:
                        # Define the expected full feature size
                        # MediaPipe hands has 21 landmarks per hand with x,y,z (21*3=63 per hand)
                        # MediaPipe pose has 33 landmarks with x,y,z (33*3=99)
                        # Maximum expected: 2 hands (126) + pose (99) = 225 features
                        
                        # Initialize arrays with zeros
                        left_hand_features = np.zeros(63, dtype=np.float32)  # 21 points * 3 coords
                        right_hand_features = np.zeros(63, dtype=np.float32) # 21 points * 3 coords
                        pose_features = np.zeros(99, dtype=np.float32)       # 33 points * 3 coords
                        
                        # Extract hand keypoints if present
                        if 'hands' in frame and frame['hands']:
                            for i, hand in enumerate(frame['hands']):
                                # Each hand has 21 keypoints with x,y,z coordinates
                                if i < 2:  # Only process up to 2 hands
                                    hand_features = []
                                    for point in hand:
                                        if isinstance(point, list) and len(point) == 3:
                                            hand_features.extend(point)  # Add x,y,z for each keypoint
                                    
                                    # Assign to left or right hand array (assuming first hand is left)
                                    if i == 0 and len(hand_features) <= 63:
                                        left_hand_features[:len(hand_features)] = hand_features
                                    elif i == 1 and len(hand_features) <= 63:
                                        right_hand_features[:len(hand_features)] = hand_features
                        
                        # Extract pose keypoints if present
                        if 'pose' in frame and frame['pose']:
                            pose_data = []
                            for point in frame['pose']:
                                if isinstance(point, list) and len(point) == 3:
                                    pose_data.extend(point)  # Add x,y,z for each keypoint
                            
                            # Ensure we don't exceed the expected size
                            if len(pose_data) <= 99:
                                pose_features[:len(pose_data)] = pose_data
                        
                        # Combine all features into a single consistent vector
                        frame_features = np.concatenate([left_hand_features, right_hand_features, pose_features])
                        processed_frames.append(frame_features)
                    
                    # Only add sequences with features
                    if processed_frames:
                        processed_data = np.array(processed_frames, dtype=np.float32)
                        
                        # Additional validation
                        if processed_data.shape[0] > 0 and processed_data.shape[1] > 0:
                            data.append(processed_data)
                            labels.append(label)
                            processed_files += 1
                        else:
                            print(f"Warning: Invalid data shape in {json_file}: {processed_data.shape}")
                            skipped_files += 1
                    else:
                        print(f"Warning: No valid frames found in {json_file}")
                        skipped_files += 1
                        
            except Exception as e:
                print(f"Error processing {json_file}: {str(e)}")
                skipped_files += 1
    
    print(f"Processed {processed_files} files successfully, skipped {skipped_files} files")
    if not data:
        print("Warning: No valid data was loaded!")
        return np.array([]), np.array([])
        
    # Make sure all samples have the same shape for each sequence dimension
    # Find the maximum sequence length
    max_seq_len = max(sample.shape[0] for sample in data)
    feature_dim = data[0].shape[1]  # Should be 225 (63+63+99)
    
    # Create a padded dataset with uniform dimensions
    padded_data = []
    for sample in data:
        # If sequence is shorter than max_len, pad with zeros
        if sample.shape[0] < max_seq_len:
            padding = np.zeros((max_seq_len - sample.shape[0], feature_dim), dtype=np.float32)
            padded_sample = np.vstack((sample, padding))
        else:
            # If longer, truncate
            padded_sample = sample[:max_seq_len]
        padded_data.append(padded_sample)
    
    return np.array(padded_data), np.array(labels)

# Load data
data, labels = load_keypoints(DATA_DIR)

Loading keypoints from f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints


Loading labels: 0it [00:00, ?it/s]

Found 8 files in about


Loading labels: 1it [00:00,  4.66it/s] [00:00<?, ?it/s]

Found 13 files in accident


...

Loading labels: 194it [01:06,  2.28it/s] [00:00<?, ?it/s]

Found 7 files in movie


Loading labels: 195it [01:06,  2.61it/s]00:00<?, ?it/s]

Found 10 files in music


Processing music:   0%|          | 0/10 [00:00<?, ?it/s]

# Dataset Preparation: Train-Test Split

Split the dataset into training, validation, and test sets.


In [ ]:
# Split the dataset
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, stratify=labels, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(f"Training set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")

In [ ]:
# Define the dataset class
class ASLDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Create the datasets
train_dataset = ASLDataset(X_train, y_train)
val_dataset = ASLDataset(X_val, y_val)

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model Definition: BiLSTM for Temporal Data

Define a BiLSTM model for ASL recognition.


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Print information about the data shape
print(f"Data shape: {data.shape}")
if len(data) > 0:
    print(f"Each sample has {data[0].shape[0]} frames with {data[0].shape[1]} features per frame")

# Define the model with Batch Normalization
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size * 2)
        self.dropout1 = nn.Dropout(0.3)
        self.lstm2 = nn.LSTM(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size * 2)
        self.dropout2 = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.batch_norm1(x.transpose(1, 2)).transpose(1, 2)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.batch_norm2(x.transpose(1, 2)).transpose(1, 2)
        x = self.dropout2(x)
        x = self.fc(x[:, -1, :])
        return x

# Get the input size from the processed data
input_size = data[0].shape[1] if len(data) > 0 else 0
print(f"Input size for the model: {input_size}")
hidden_size = 128
output_size = len(np.unique(labels))
print(f"Number of unique labels (classes): {output_size}")

model = BiLSTM(input_size, hidden_size, output_size)
print(model)
# Move model to selected device
model = model.to(device)

# Hyperparameter Tuning

Tune Hyperparams.


In [ ]:
# Hyperparameter optimization with Optuna
import optuna

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameters to search
    lr = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 256])
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    
    # Create a new model with trial hyperparameters
    model_trial = BiLSTM(input_size, hidden_size, output_size).to(device)
    
    # Update dropout rates based on trial
    model_trial.dropout1 = nn.Dropout(dropout_rate)
    model_trial.dropout2 = nn.Dropout(dropout_rate)
    
    # Setup optimizer
    optimizer_trial = optim.Adam(model_trial.parameters(), lr=lr)
    criterion_trial = nn.CrossEntropyLoss()
    
    # Create dataloaders with trial batch size
    train_loader_trial = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader_trial = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Training loop with early stopping
    num_epochs = 10  # Fewer epochs for tuning
    patience = 3
    best_val_loss = float('inf')
    early_stop_counter = 0
    
    for epoch in range(num_epochs):
        model_trial.train()
        train_loss = 0
        for batch in train_loader_trial:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer_trial.zero_grad()
            outputs = model_trial(inputs)
            loss = criterion_trial(outputs, targets)
            loss.backward()
            optimizer_trial.step()
            train_loss += loss.item()
        train_loss /= len(train_loader_trial)
        
        model_trial.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader_trial:
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model_trial(inputs)
                loss = criterion_trial(outputs, targets)
                val_loss += loss.item()
        val_loss /= len(val_loader_trial)
        
        # Report intermediate metric to Optuna
        trial.report(val_loss, epoch)
        
        # Handle pruning (early stopping for this trial)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                break
    
    return best_val_loss

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20, timeout=3600)  # Run for max of 20 trials or 1 hour

# Print optimization results
print("\nBest trial:")
trial = study.best_trial
print(f"  Value (validation loss): {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Store best parameters for model training
best_params = {
    'learning_rate': trial.params['learning_rate'],
    'hidden_size': trial.params['hidden_size'],
    'dropout_rate': trial.params['dropout_rate'],
    'batch_size': trial.params['batch_size']
}

# Visualize the optimization history
plt.figure(figsize=(10, 6))
optuna.visualization.matplotlib.plot_optimization_history(study)
plt.title('Optimization History')
plt.show()

# Visualize parameter importances
plt.figure(figsize=(10, 6))
optuna.visualization.matplotlib.plot_param_importances(study)
plt.title('Parameter Importances')
plt.show()

# Model Training

Train the model on the training set and validate on the validation set.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Use best hyperparameters from Optuna tuning
best_lr = best_params['learning_rate']
best_hidden_size = best_params['hidden_size']
best_dropout_rate = best_params['dropout_rate']
best_batch_size = best_params['batch_size']

print(f"Training with best hyperparameters:")
print(f"  Learning rate: {best_lr}")
print(f"  Hidden size: {best_hidden_size}")
print(f"  Dropout rate: {best_dropout_rate}")
print(f"  Batch size: {best_batch_size}")

# Recreate model with best hyperparameters
model = BiLSTM(input_size, best_hidden_size, output_size).to(device)
model.dropout1 = nn.Dropout(best_dropout_rate)
model.dropout2 = nn.Dropout(best_dropout_rate)

# Define the loss function and optimizer with best learning rate
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=best_lr)

# Create data loaders with best batch size
train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)

# Training with Early Stopping
train_losses = []
val_losses = []
patience = 5
early_stop_counter = 0
best_val_loss = float('inf')
epochs = 20
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    # Save the model if validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print("Model checkpoint saved!")
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        print(f"Early stopping counter: {early_stop_counter}/{patience}")
    if early_stop_counter >= patience:
        print("Early stopping triggered.")
        break

In [ ]:
# Plot Training and Validation Loss Curves
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Curves")
plt.legend()
plt.show()

# Evaluation

Evaluate the model on the test set and display metrics.


In [ ]:
# Evaluate the model
test_dataset = ASLDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)
model.eval()
test_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)  # Move inputs to device too
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)
test_loss /= len(test_loader)
accuracy = correct / total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Save the model for transfer learning
os.makedirs('./models', exist_ok=True)  # Create directory if it doesn't exist
torch.save({
    'model_state_dict': model.state_dict(),
    'input_size': input_size,
    'hidden_size': best_hidden_size,
    'dropout_rate': best_dropout_rate,
    'output_size': output_size,
    'accuracy': accuracy
}, "./models/transfer_learning_model.pth")
print("Model saved for transfer learning!")

# Inference

Test the model on new data or perform real-time inference.


In [ ]:
# Example inference
model.eval()
sample = torch.tensor(X_test[0:1], dtype=torch.float32).to(device)  # Move sample to the correct device
with torch.no_grad():
    prediction = model(sample)
    predicted_label = torch.argmax(prediction, dim=1).item()
print(f"Predicted label: {predicted_label}, True label: {y_test[0]}")